# ContextualCompressionRetriever

In [2]:
from langchain_core.documents import Document
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]

In [3]:
from langchain_community.vectorstores import Chroma
import os

from langchain.embeddings import HuggingFaceEmbeddings
huggingfacehub_api_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


C:\Users\ANKIT\AppData\Local\Temp\ipykernel_7684\2258002492.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
f:\study\LangGraph_LangChain\llmenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(
    documents=docs,
    embedding=embedding_model
)

In [6]:
base_retriever = vectorstore.as_retriever(search_kwargs={"k":5})

In [8]:
import os
from langchain_groq import ChatGroq
from langchain.schema import Document

os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")
model = ChatGroq(model="llama3-70b-8192")

In [12]:
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever

In [10]:
compressor = LLMChainExtractor.from_llm(llm=model)

In [11]:
compressor

LLMChainExtractor(llm_chain=PromptTemplate(input_variables=['context', 'question'], input_types={}, output_parser=NoOutputParser(), partial_variables={}, template='Given the following question and context, extract any part of the context *AS IS* that is relevant to answer the question. If none of the context is relevant return NO_OUTPUT. \n\nRemember, *DO NOT* edit the extracted parts of the context.\n\n> Question: {question}\n> Context:\n>>>\n{context}\n>>>\nExtracted relevant parts:')
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000023DB78D4BF0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000023DB790F0B0>, model_name='llama3-70b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))
| NoOutputParser(), get_input=<function default_get_input at 0x0000023DF57A19E0>)

In [14]:
compressor_retirever = ContextualCompressionRetriever(base_compressor=compressor,base_retriever=base_retriever)

In [15]:
query = "what is photosynthesis"

In [17]:
result = compressor_retirever.invoke(query)

In [18]:
for i, doc in enumerate(result):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Photosynthesis is the process by which green plants convert sunlight into energy.

--- Result 2 ---
The chlorophyll in plant cells captures sunlight during photosynthesis.

--- Result 3 ---
Photosynthesis does not occur in animal cells.
